<a href="https://colab.research.google.com/github/datacorner/Covid19/blob/main/Get_Meteo_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import lxml.html as lh
from datetime import datetime, timedelta

Récupérer les données de
https://www.historique-meteo.net/france/ile-de-france/2020/01/  
Par jour https://www.historique-meteo.net/france/alsace/2020/01/01/

In [ ]:
refs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/region_meteo.csv")
refs.head(3)

,region,url,Tempmax,Tempmin,Humidite,Dureejour,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,alsace,https://www.historique-meteo.net/france/alsace/,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...",NaN,NaN,NaN,0
1,aquitaine,https://www.historique-meteo.net/france/aquita...,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...",NaN,NaN,NaN,1
2,ardeche,https://www.historique-meteo.net/france/ardeche/,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...",NaN,NaN,NaN,2


In [ ]:
# lecture directe dans drive
# sheet_url = "https://docs.google.com/spreadsheets/d/1aTk22MyHN57VE8oXUTwKkT8h6d6DD_SA-F_TqKibSX0/edit#gid=362280831"
# url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
# print(url_1)
# refs = pd.read_csv(url_1)
# refs.head(3)

In [ ]:
refs['region'][0]

'alsace'

# Get data from each page (per region and per day)

In [ ]:
def getTemp(_val):
  try:
    myval = _val.replace('Â', '').replace('°', '')
    return int(myval)
  except:
    return -1

def getWet(_val):
  try:
    myval = _val.replace('%', '')
    return int(myval)
  except:
    return -1

def getSunTimeInSec(_val):
  try:
    vals = _val.split(':')
    return int(vals[2]) + int(vals[1]) * 60 +  int(vals[0]) * 60 * 60
  except:
    return -1

# Return the text value from an XPath
def getValueFromXPath(doc, _xpath):
  try:
    value = doc.xpath(_xpath)
    return value[0].text_content().strip()
  except:
    return "Error"

# Return the meteo data for 1 day / 1 region
def getRegionMeteofromDay(_index, _day):
  page = requests.get(refs['url'][_index] + _day)
  doc = lh.fromstring(page.content)
  tempmax = getValueFromXPath(doc, refs['Tempmax'][_index])
  tempmin = getValueFromXPath(doc, refs['Tempmin'][_index])
  humidite = getValueFromXPath(doc, refs['Humidite'][_index])
  dureejour = getValueFromXPath(doc, refs['Dureejour'][_index])
  return getTemp(tempmax), getTemp(tempmin), getWet(humidite), getSunTimeInSec(dureejour)

In [ ]:
tmax, tmin, wet, timeduration = getRegionMeteofromDay(1, "2020/01/01/")
print (tmax, tmin, wet, timeduration)

-1 -1 -1 -1


# Loop per day for all region

Create a CSV file for each region for all days

In [ ]:
# Counter date function
starting_date = datetime(2020, 1, 1)
today = datetime.now()

def formatDateForURL(_date):
  return (str(_date.year).zfill(4) + "/" + str(_date.month).zfill(2) + "/" + str(_date.day).zfill(2)) 

def addDate(_counteridx):
  return formatDateForURL(starting_date + timedelta(_counteridx))

#sending_date = formatDateForURL(today)
sending_date = '2020/12/01'
sstarting_date = formatDateForURL(starting_date)

In [ ]:

for indexRegion in range (22):
  reg1 = pd.DataFrame()
  i = 0
  date = sstarting_date

  print("Start:", refs['region'][indexRegion])
  while date != sending_date:
    if not(i % 100):
      print(f"Date: {date} / index: {i}")
    date = addDate(i)
    tmax, tmin, wet, timeduration = getRegionMeteofromDay(indexRegion, date)
    reg1 = reg1.append([(refs['region'][indexRegion], date, tmax, tmin, wet, timeduration)])
    i = i + 1

  # Change columns names
  reg1.columns = ['region', 'date', 'tmax', 'tmin', 'wet', 'suntime']
  reg1.to_csv(refs['region'][indexRegion] + '.csv')

# Agregate the region files for temperatures